In [1]:
import pandas as pd
import numpy as np
import random
import joblib
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
# SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)

In [3]:
# df = pd.read_csv('../../data/anemia_synth_dataset_hb_with_unspecified.csv')
# #df = df.fillna(-1)
# df.head()


In [4]:
class_dict = constants.CLASS_DICT

#### Creating the anemia dataset

In [5]:
anem_feat_arr = [{'name':'hemoglobin', 'lower':12.1, 'upper':17.2, 'abnorm_lower':6, 'abnorm_upper':18},
                 {'name':'ferritin', 'lower':10, 'upper':263, 'abnorm_lower':0, 'abnorm_upper': 500},
                 {'name':'ret_count', 'lower':0.5, 'upper':2.5, 'abnorm_lower':0, 'abnorm_upper':6},
                 {'name':'segmented_neutrophils', 'lower':0, 'upper':0, 'abnorm_lower':0.01, 'abnorm_upper':7},
                 {'name':'tibc', 'lower':250, 'upper':450, 'abnorm_lower': 100, 'abnorm_upper':500},
                 {'name':'mcv', 'lower':80, 'upper':100, 'abnorm_lower': 75, 'abnorm_upper':105},
                 {'name':'gender', 'lower':0, 'upper':1}, #0=Female 1=male
                ]

In [6]:
add_feat_arr = [{'name':'serum_iron', 'lower':60, 'upper':170, 'abnorm_lower':20, 'abnorm_upper':250},
                {'name':'rbc', 'lower':3.8, 'upper':5.9, 'abnorm_lower': 2.5, 'abnorm_upper':7},
                {'name':'age', 'lower':20, 'upper':90, 'abnorm_lower':18, 'abnorm_upper':95},
                {'name':'indirect_bilirubin', 'lower':0.2, 'upper':1.2, 'abnorm_lower':0, 'abnorm_upper':3},
                {'name':'transferrin', 'lower':204, 'upper':360, 'abnorm_lower':150, 'abnorm_upper':450},
                {'name':'creatinine', 'lower':0.6, 'upper':1.3, 'abnorm_lower':0.2, 'abnorm_upper':2},
                {'name':'cholestrol', 'lower':30, 'upper':100, 'abnorm_lower':0, 'abnorm_upper':150},
                {'name':'copper', 'lower':62, 'upper':140, 'abnorm_lower':30, 'abnorm_upper':130},
                {'name':'ethanol', 'lower':0, 'upper':50, 'abnorm_lower':0, 'abnorm_upper':80},
                {'name':'folate', 'lower':2.7, 'upper':17, 'abnorm_lower':0.5, 'abnorm_upper':30},
                {'name':'glucose', 'lower':70, 'upper':100, 'abnorm_lower':40, 'abnorm_upper':140}
               ]

In [7]:
total_feat_arr = anem_feat_arr + add_feat_arr

In [8]:
def create_label(row):
    if row['hemoglobin']> 13:
        return 'No anemia'
    elif (row['hemoglobin'] > 12) & (row['gender']==0):
        return 'No anemia'
    else:
        if row['mcv']<80 :
            if row['ferritin']<30:
                return 'Iron deficiency anemia'
            elif row['ferritin']>100:
                return 'Anemia of chronic disease'
            elif row['tibc']<450:
                return 'Anemia of chronic disease'
            elif row['tibc']>=450:
                return 'Iron deficiency anemia'
            else:
                return 'Inconclusive diagnosis'
            
        elif row['mcv']<=100:
            if row['ret_count'] <= 2:
                return 'Aplastic anemia'
            elif row['ret_count'] >2:
                return 'Hemolytic anemia'
            else:
                return 'Inconclusive diagnosis'

        elif row['mcv']> 100:
            if row['segmented_neutrophils']>0:
                return 'Vitamin B12/Folate deficiency anemia'
            elif row['segmented_neutrophils']==0:
                return 'Unspecified anemia'
            else:
                return 'Inconclusive diagnosis'
        else:
            return 'Inconclusive diagnosis'

In [9]:
def uniform_dist(df, feat_dict, num=None):
    col_name = feat_dict['name']
    col_values = np.random.uniform(feat_dict['abnorm_lower'], feat_dict['abnorm_upper'], num)
    return col_values

In [10]:
def create_dataset(sample_num, dist_type='uniform'):
    dataset = pd.DataFrame()
    for feat_dict in total_feat_arr:
        if feat_dict['name'] == 'gender':
            dataset[feat_dict['name']] = np.random.choice([feat_dict['lower'], feat_dict['upper']], sample_num)
        elif dist_type=='normal':
            dataset[feat_dict['name']] = normal_dist(dataset, feat_dict, sample_num)
        else:
            dataset[feat_dict['name']] = uniform_dist(dataset, feat_dict, sample_num)
    return dataset

In [11]:
def add_nans(df):
    for col in df.columns:
        if col not in ['gender', 'age', 'hemoglobin', 'mcv']:
            col_frac = round(random.uniform(0.1, 0.7), 1)
            #df.loc[df.sample(frac=col_frac, random_state=SEED).index, col] = np.nan
            df.loc[df.sample(frac=col_frac).index, col] = np.nan
    return df

In [12]:
ft_df = create_dataset(60000)
ft_df = add_nans(ft_df)
#add_ft_df = add_ft_df.fillna(-1)
ft_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,gender,serum_iron,rbc,age,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
0,10.494481,NaN,0.778524,NaN,130.054397,79.711619,0,196.452817,5.682914,68.316876,2.803736,157.777445,1.479905,81.057067,94.471461,40.485653,24.584653,66.535764
1,17.408572,440.550938,3.238355,5.264643,NaN,77.865263,0,50.130228,3.117206,27.718570,NaN,347.301650,NaN,82.768328,88.526456,66.435307,11.769449,99.248629
2,14.783927,NaN,3.692318,2.047316,366.583996,79.138176,1,NaN,3.525492,49.089143,1.101304,201.139144,NaN,81.271119,56.099047,42.779875,10.374046,78.882162
3,13.183902,144.589366,3.046232,6.275160,222.126370,89.204682,0,154.463299,3.588404,42.461004,1.498549,258.928986,NaN,5.160539,113.620935,NaN,16.463197,NaN
4,7.872224,NaN,3.105583,2.775346,329.663388,101.536021,1,NaN,NaN,64.087142,0.001965,NaN,1.772483,89.690830,NaN,30.464260,24.605084,90.364160


#### Concatenate the dfs

In [13]:
# new_df = pd.concat([df, add_ft_df], axis=1)
new_df = ft_df
# new_df['new_label'] = new_df.apply(lambda row: create_label(row), axis=1)
new_df['label'] = new_df.apply(lambda row: create_label(row), axis=1)
#new_df = new_df.drop(['label'], axis=1)
#new_df = new_df.rename({'new_label': 'label'}, axis=1)
new_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,gender,serum_iron,rbc,age,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,10.494481,NaN,0.778524,NaN,130.054397,79.711619,0,196.452817,5.682914,68.316876,2.803736,157.777445,1.479905,81.057067,94.471461,40.485653,24.584653,66.535764,Anemia of chronic disease
1,17.408572,440.550938,3.238355,5.264643,NaN,77.865263,0,50.130228,3.117206,27.718570,NaN,347.301650,NaN,82.768328,88.526456,66.435307,11.769449,99.248629,No anemia
2,14.783927,NaN,3.692318,2.047316,366.583996,79.138176,1,NaN,3.525492,49.089143,1.101304,201.139144,NaN,81.271119,56.099047,42.779875,10.374046,78.882162,No anemia
3,13.183902,144.589366,3.046232,6.275160,222.126370,89.204682,0,154.463299,3.588404,42.461004,1.498549,258.928986,NaN,5.160539,113.620935,NaN,16.463197,NaN,No anemia
4,7.872224,NaN,3.105583,2.775346,329.663388,101.536021,1,NaN,NaN,64.087142,0.001965,NaN,1.772483,89.690830,NaN,30.464260,24.605084,90.364160,Vitamin B12/Folate deficiency anemia


In [14]:
new_df.isna().sum()

hemoglobin                   0
ferritin                 30000
ret_count                 6000
segmented_neutrophils    18000
tibc                     12000
mcv                          0
gender                       0
serum_iron               30000
rbc                      30000
age                          0
indirect_bilirubin       36000
transferrin              12000
creatinine               24000
cholestrol                6000
copper                   12000
ethanol                  24000
folate                    6000
glucose                  12000
label                        0
dtype: int64

In [15]:
new_df.label.value_counts()

No anemia                               27529
Hemolytic anemia                        13114
Aplastic anemia                          6509
Inconclusive diagnosis                   4428
Anemia of chronic disease                4230
Vitamin B12/Folate deficiency anemia     3733
Iron deficiency anemia                    457
Name: label, dtype: int64

In [16]:
# new_df.to_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv', index=False)